思路：先将FGSM和PGD，引入PGD的有目标攻击，再将整数规划问题优化的有目标攻击

# 0. 课程前言

此为《人工智能安全》实验课第三部分：可认证鲁棒性实验部分.

可认证鲁棒性，即在面对对抗样本的时候，找到一个可以量化的界限，即明确一个标准，以判断系统在面对恶意干扰或异常状况时的表现程度，这个可量化的界限可以帮助研究者评估模型的鲁棒程度，并采取相应的改进措施。

回顾对抗攻击实验部分，其核心目标是寻找使模型分类失效的最小对抗扰动。具体而言，对抗攻击可形式化为如下优化问题：

$$
\delta^* = \arg \max _{||\delta|| \le \epsilon} \mathcal L (h_\theta(x), y)
$$


其中，扰动 $\delta$ 的优化解域存在显著差异性：部分解会导致模型以高概率误判（对抗攻击上界），而另一部分解仅能引发低概率误判（对抗攻击下界）。这种优化目标的上下界分析构成了对抗攻防研究的核心框架。

基于此，我们将对抗攻击方法系统性地划分为三类研究范式：
1. **启发式局部优化**

    找到优化目标 $\delta$ 的下界，即"通过启发式算法找到一个对抗样本"。主流方法如 **FGSM(快速梯度符号法)** 和 **PGD(投影梯度下降法)** 等，通过经验驱动的局部搜索策略构造对抗样本。这类方法本质上是对非凸优化问题的近似求解，虽能有效发现对抗样本，但无法保证获得全局最优解，属于对抗攻击下界的探索。
2. **精确全局优化**

    针对特定网络结构（如分段线性激活函数），将对抗样本构造问题转化为组合优化问题，利用混合整数规划（MIP）等数学工具进行精确求解。该方法能够确定性地验证是否存在导致误判的对抗样本，但受限于计算复杂度，主要适用于小型网络验证。
3. **可认证鲁棒性分析**

    通过构建网络结构的凸松弛模型（如线性规划松弛、半正定规划松弛），在更易处理的优化空间内推导对抗攻击上界。当松弛模型在给定扰动范围内无法构造成功攻击时，即可严格证明原始网络具有可认证鲁棒性。这种基于数学证明的方法为模型安全性提供了理论保证。

在对抗训练实验部分，我们已初步引入可认证鲁棒性分析的相关内容，本次实验我们会进行更深入的介绍。

# 1. 训练准备

我们先引入三个简单的神经网络模型：

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)    

model_dnn_2 = nn.Sequential(Flatten(), nn.Linear(784,200), nn.ReLU(), 
                            nn.Linear(200,10)).to(device)

model_dnn_4 = nn.Sequential(Flatten(), nn.Linear(784,200), nn.ReLU(), 
                            nn.Linear(200,100), nn.ReLU(),
                            nn.Linear(100,100), nn.ReLU(),
                            nn.Linear(100,10)).to(device)

model_cnn = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
                          nn.Conv2d(32, 32, 3, padding=1, stride=2), nn.ReLU(),
                          nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
                          nn.Conv2d(64, 64, 3, padding=1, stride=2), nn.ReLU(),
                          Flatten(),
                          nn.Linear(7*7*64, 100), nn.ReLU(),
                          nn.Linear(100, 10)).to(device)

In [4]:
model_dnn_2.load_state_dict(torch.load("model_dnn_2.pt"))
model_dnn_4.load_state_dict(torch.load("model_dnn_4.pt"))
model_cnn.load_state_dict(torch.load("model_cnn.pt"))

/tmp/ipykernel_468744/811293303.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dnn_2.load_state_dict(torch.load("model_dnn_2.pt"))
/tmp/ipykernel_468744/811293303

<All keys matched successfully>

对于一个神经网络，可以采用如下方式进行定义：
$$
\begin{align}
z_1 &= x \notag \\
z_{i+1} &= f_i(W_i z_i + b_i), i = 1, 2, \cdots, d \notag \\
h_\theta(x) &= z_{d+1} \notag
\end{align}
$$
通过上述递推关系，描述了神经网络从输入 $x$ 开始，历经一系列由权重矩阵 $W_i$、偏置向量 $b_i$ 以及激活函数 $f_i$ 共同作用的计算过程，最终输出 $h_\theta(x)$ 作为网络对输入 $x$ 的处理结果。

首先考虑如何将有针对性的对抗性攻击构建为一个优化问题。有针对性的攻击将尝试最小化真实类别 $h_\theta(x+\delta, y)$ 的对数几率，并最大化目标类别 $h_\theta(x+\delta, y_{targ})$ 的对数几率；

以 ReLu 作为网络激活函数，我们可以将目标写成如下带有约束的优化问题：

$$
\begin{align}
\arg \min _{z_1, z_2, \cdots, z_{d+1}} \; &(e_y - e_{y_{targ}})^T z_{d+1} \notag \\
subject \; \; to \; &||z_1 - x||_\infty \le \epsilon \notag \\
&z_{i+1} = max \{0, W_i z_i + b_i\}, i = 1, 2, \cdots, d-1 \notag \\
&z_{d+1} = W_d z_d + b_d \notag
\end{align}
$$

这里的 $e_i$ 表示单位基，即一个在第 $i$ 个位置为 1，其他位置为 0 的向量。

我们去掉了显式的 $\delta$ 项，采用了一个对 $z_1$ （即第一层的对抗样本输入）在原始输入 $x$ 的 $\epsilon$ 范围内的约束。

这里的优化问题的表述并不是求解器实际能解决的形式，因为涉及到最大运算符的等式约束，它既不是凸约束，也不是大多数优化求解器原生处理的约束。

为了能将其表述为能被求解器解决的问题，我们需要将其转化为另一种形式：二进制混合整数线性规划（MILP）。
> 二进制 MILP 是一个优化问题，由线性目标、线性等式和不等式约束以及对一些变量 $z_i \in \{0, 1\}$的二进制约束组成。一般来说，MILP是 NP 难题，所以不期望将其扩展到现代神经网络规模，但对于小问题，MILP是一个广泛研究的领域，并且存在比简单地尝试所有二进制变量设置的朴素暴力方法更具可扩展性的方法。

假设有 $z_{i+1} = max\{0, W_i z_i + b_i\}$ 可能取值的一些已知上下界（由 $l_i$ 和 $u_i$ 表示），我们引入一组与 $z_{i+1}$ 形状相同的二进制变量 $v_i$ ，我们可用以下不等式来等价于约束条件 $z_{i+1} = max\{0, W_i z_i + b_i\}$：

$$
\begin{align}
z_{i+1} &\ge W_i z_i + b_i \notag \\
z_{i+1} &\ge 0 \notag \\
u_i \cdot v_i &\ge z_{i+1} \notag \\
W_i z_i + b_i &\ge z_{i+1} + (1 - v_i) l_i \notag \\
v_i &\in \{0, 1\}^{|v_i|} \notag
\end{align}
$$

此时的问题就变成了求解 $l_i$ 和 $u_i$ ，受求解器的求解时间限制，不能简单地令其为 $-10^100$ 和 $10^100$ 这样显然存在的解。但我们实际上可以用与计算有目标攻击完全相同的方式精确计算这些界的值：通过最小化中间层的单个神经元的激活值来计算上界或下界。

$$
\begin{align}
\arg \min_{z_1, z_2, \cdots, z_{k+1}} &e_j^T z_{k+1} \notag \\
subject \; \; to \; &||z_1 - x||_\infty \le \epsilon \notag \\
&z_{i+1} = max\{0, W_i z_i + b_i\}, i = 1, 2, \cdots, k-1 \notag \\
&z_{k+1} = W_k z_k + b_k \notag
\end{align}
$$

虽然上述公式能求解上下界的精确解，但这样我们实际上是在为网络中的每个隐藏单元求解两个整数规划问题，所以这是极其不切实际的。
更实际的做法是简单地选取一组更加宽松（但又不如同 $10^{100}$ 那样完全“空洞”）且计算速度更快的边界。

一个自然的选择是使用简单的区间边界，对于某一层而言，前一层的输出 $x$ 存在一些边界 $\tilde l \le x \le \tilde u$，那么，这一层的 $z$ 存在的上下界即为使得 $z = Wx + b$ 的最大或最小值。

考虑 $z$ 所在层的第 $i$ 个神经元，有

$$
(Wz + b)_i = \sum_j w_{ij}x_j + b_i
$$

显然，求这一层的下界，需采用如下策略：当 $W_{ij}$ 为负时，应该选取 $x_j = \tilde u_j$ 这个上界；当 $W_{ij}$ 为正时，应该选取 $x_j = \tilde l_j$ 这个下界。（若求层的上界，则反之亦然）。因此，我们得到了 $z = Wx + b$ 的边界集合，有以下条件给出：

$$
max\{W, 0\} \tilde l + min \{W, 0\} \tilde u + b \le (Wx + b) \le max\{W, 0\}\tilde u + min \{W, 0\} \tilde l + b
$$

需要注意的是，这个上下界并不是精确的，而是一组更宽松的界限。

下面，我们在神经网络模型上计算每一层的区间边界：

In [ ]:
def bound_propagation(model, initial_bound):
    # 传入参数：initial_bound 为初始输入的扰动区间，即 [x - epsilon, x + epsilon]
    l, u = initial_bound
    bounds = []
    
    for layer in model:
        if isinstance(layer, Flatten):
            l_ = Flatten()(l)
            u_ = Flatten()(u)
        elif isinstance(layer, nn.Linear):
            l_ = (layer.weight.clamp(min=0) @ l.t() + layer.weight.clamp(max=0) @ u.t() 
                  + layer.bias[:,None]).t()
            u_ = (layer.weight.clamp(min=0) @ u.t() + layer.weight.clamp(max=0) @ l.t() 
                  + layer.bias[:,None]).t()
        elif isinstance(layer, nn.Conv2d):
            l_ = (nn.functional.conv2d(l, layer.weight.clamp(min=0), bias=None, 
                                       stride=layer.stride, padding=layer.padding,
                                       dilation=layer.dilation, groups=layer.groups) +
                  nn.functional.conv2d(u, layer.weight.clamp(max=0), bias=None, 
                                       stride=layer.stride, padding=layer.padding,
                                       dilation=layer.dilation, groups=layer.groups) +
                  layer.bias[None,:,None,None])
            
            u_ = (nn.functional.conv2d(u, layer.weight.clamp(min=0), bias=None, 
                                       stride=layer.stride, padding=layer.padding,
                                       dilation=layer.dilation, groups=layer.groups) +
                  nn.functional.conv2d(l, layer.weight.clamp(max=0), bias=None, 
                                       stride=layer.stride, padding=layer.padding,
                                       dilation=layer.dilation, groups=layer.groups) + 
                  layer.bias[None,:,None,None])
            
        elif isinstance(layer, nn.ReLU):
            l_ = l.clamp(min=0)
            u_ = u.clamp(min=0)
            
        bounds.append((l_, u_))
        l,u = l_, u_
    return bounds

接下来，我们输出模型最后一层在第一个样本("0"图像)上的上下界信息：

In [ ]:
epsilon = 0.1
bounds = bound_propagation(model_cnn, ((X - epsilon).clamp(min=0), (X + epsilon).clamp(max=1)))
# print: 模型最后一层的第一个样本的下界信息
print("lower bound: ", bounds[-1][0][0].detach().cpu().numpy())
print("upper bound: ", bounds[-1][1][0].detach().cpu().numpy())

lower bound:  [-2855.4631 -2571.039  -3696.8293 -2543.9111 -2867.7935 -2444.5105
 -2912.023  -2475.7058 -2545.2742 -2578.3105]
upper bound:  [2186.5938 2673.1711 2457.4033 2726.826  2844.7742 3080.524  2242.0413
 3211.849  3151.519  2782.5747]


In [36]:
epsilon = 0.1
bounds = bound_propagation(model_dnn_2, ((X - epsilon).clamp(min=0), (X + epsilon).clamp(max=1)))
print("lower bound: ", bounds[-1][0][0].detach().cpu().numpy())
print("upper bound: ", bounds[-1][1][0].detach().cpu().numpy())

lower bound:  [-20.560423 -23.64547  -17.535437 -16.681828 -32.128284 -22.724869
 -30.491901 -15.737773 -23.58483  -25.076939]
upper bound:  [19.24827  14.374581 25.719341 29.486425 18.704813 28.60849  11.866371
 32.1335   23.711868 25.589342]


In [37]:
epsilon = 0.1
bounds = bound_propagation(model_dnn_4, ((X - epsilon).clamp(min=0), (X + epsilon).clamp(max=1)))
print("lower bound: ", bounds[-1][0][0].detach().cpu().numpy())
print("upper bound: ", bounds[-1][1][0].detach().cpu().numpy())

lower bound:  [-220.44913 -206.34962 -187.77368 -221.78787 -233.58583 -179.13441
 -244.60938 -202.73329 -207.64474 -204.78436]
upper bound:  [192.9366  211.39684 180.75684 212.42493 194.95622 216.43507 192.93724
 239.62477 209.22014 220.56755]


# 在整数规划中使用这些界限

上述的整数规划问题已经解决了上下界的范围问题，现在，我们使用求解器来求解这个整数规划问题。

我们把上述所有整数规划约束结合起来，指定了如下最终的整数规划公式：

$$
\begin{align}
\arg \min _{z_1, z_2, \cdots, z_{d+1}, v_1, v_2, v_{d-1}} &{e_y - e_{y_{targ}}}^T z_{d+1} \notag \\
subject \; \; to \;  &z_{i+1} \ge W_i z_i + b_i, i = 1, 2, \cdots, d-1 \notag \\
&z_{i+1} \ge 0, i = 1, 2, \cdots, d-1 \notag \\
&u_i \cdot v_i \ge z_{i+1}, i = 1, 2, \cdots, d-1 \notag \\
&W_i z_i + b_i \ge z_{i+1} + (1 - v_1) l_i, i=1, 2, \cdots, d-1 \notag \\
&v_i \in \{0, 1\}^{|v_i|} , i = 1, 2, \cdots, d-1 \notag \\
&z_1 \le x + \delta \notag \\
&z_1 \ge x - \delta  \notag \\
&z_{d+1} = W_d z_d + b_d \notag
\end{align}
$$

本实验中，我们使用 cvxpy 库中的 Gurobi 求解器来求解这个整数规划。

In [ ]:
import cvxpy as cp

# 定义线性约束
def form_milp(model, c, initial_bounds, bounds):
    linear_layers = [(layer, bound) for layer, bound in zip(model,bounds) if isinstance(layer, nn.Linear)]
    d = len(linear_layers)-1
    
    # create cvxpy variables
    z = ([cp.Variable(layer.in_features) for layer, _ in linear_layers] + 
         [cp.Variable(linear_layers[-1][0].out_features)])
    v = [cp.Variable(layer.out_features, boolean=True) for layer, _ in linear_layers[:-1]]
    
    # extract relevant matrices
    W = [layer.weight.detach().cpu().numpy() for layer,_ in linear_layers]
    b = [layer.bias.detach().cpu().numpy() for layer,_ in linear_layers]
    l = [l[0].detach().cpu().numpy() for _, (l,_) in linear_layers]
    u = [u[0].detach().cpu().numpy() for _, (_,u) in linear_layers]
    l0 = initial_bounds[0][0].view(-1).detach().cpu().numpy()
    u0 = initial_bounds[1][0].view(-1).detach().cpu().numpy()
    
    # add ReLU constraints
    constraints = []
    for i in range(len(linear_layers)-1):
        constraints += [z[i+1] >= W[i] @ z[i] + b[i], 
                        z[i+1] >= 0,
                        cp.multiply(v[i], u[i]) >= z[i+1],
                        W[i] @ z[i] + b[i] >= z[i+1] + cp.multiply((1-v[i]), l[i])]
    
    # final linear constraint
    constraints += [z[d+1] == W[d] @ z[d] + b[d]]
    
    # initial bound constraints
    constraints += [z[0] >= l0, z[0] <= u0]
    
    return cp.Problem(cp.Minimize(c @ z[d+1]), constraints), (z, v)

考虑到求解时间，我们选择一个小规模的神经网络作为求解对象。

本例中，我们将第一个图像 y[0] 作为攻击图像，其真实标签为 7 ，目标标签为 2 。

In [ ]:
plt.imshow(1-z[0].value.reshape(28,28), cmap="gray")

In [ ]:
model_small = nn.Sequential(Flatten(), nn.Linear(784,50), nn.ReLU(), 
                            nn.Linear(50,20), nn.ReLU(), 
                            nn.Linear(20,10)).to(device)

# load model from disk
model_small.load_state_dict(torch.load("model_small.pt"))

In [ ]:
initial_bound = ((X[0:1] - epsilon).clamp(min=0), (X[0:1] + epsilon).clamp(max=1))
bounds = bound_propagation(model_small, initial_bound)

# 基向量，定义原始对象和目标对象
# y[0]的真实标签为 7 ，目标标签为 2
c = np.zeros(10)
c[y[0].item()] = 1
c[2] = -1

prob, (z, v) = form_milp(model_small, c, initial_bound, bounds)

In [43]:
prob.solve(solver=cp.GUROBI, verbose=True)

                                     CVXPY                                     
                                     v1.5.2                                    
(CVXPY) Apr 12 06:26:38 PM: Your problem has 934 variables, 1858 constraints, and 0 parameters.
(CVXPY) Apr 12 06:26:38 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 12 06:26:38 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 12 06:26:38 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 12 06:26:38 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 12 06:26:38 PM: Compiling problem (target solver=GUROBI)

-3.225911939123418

求解器得到的目标是负的，这意味着我们能够找到一种扰动，使得目标类的类别的对数概率比原始类的类别的对数概率更大，即我们能够构造一个目标类别为 "2" 的对抗样本。

# 可认证鲁棒性

前面提到的基于整数规划优化问题的攻击方法，同时也给了我们验证鲁棒性的思路：对于一个给定的样本示例，我们想要知道其是否存在任何对抗样本，我们可以针对每个可能的目标类别，使用基于整数规划优化问题的攻击方法。如果这些目标类别中的任何一个有负数解，那么就存在一个目标类别的对抗样本。反之，如果对于任意目标类别，所有目标类别都不存在负数解，那么神经网络模型在这个示例上就被证明是鲁棒的。

举一个例子：对于手写数字图像识别这样的 10 分类任务，对于一张 “2” 类别的图像，我们对其使用基于整数规划的方法，以其他 9 类分别作为目标类别运行 9 次，若 “1” 类别作为目标类别存在负数解，则存在以 “1” 为目标类别的对抗样本。

让我们看看如何做到这一点，以验证较小的扰动球不能改变类别标签。

In [47]:
epsilon = 0.05
initial_bound = ((X[0:1] - epsilon).clamp(min=0), (X[0:1] + epsilon).clamp(max=1))
bounds = bound_propagation(model_small, initial_bound)

for y_targ in range(10):
    if y_targ != y[0].item():
        c = np.eye(10)[y[0].item()] - np.eye(10)[y_targ]
        prob, _ = form_milp(model_small, c, initial_bound, bounds)
        print("Targeted attack {} objective: {}".format(y_targ, prob.solve(solver=cp.GUROBI)))

Targeted attack 0 objective: 8.084621471168452
Targeted attack 1 objective: 12.975467614889116
Targeted attack 2 objective: 3.16368559925143
Targeted attack 3 objective: 2.4411197523280723
Targeted attack 4 objective: 11.65788101861424
Targeted attack 5 objective: 9.064827362176294
Targeted attack 6 objective: 22.402332700312176
Targeted attack 8 objective: 4.976967701637535
Targeted attack 9 objective: 6.069808212270474


其他 9 个类别的求解都为正数，这就说明不存在半径大小为 $\epsilon = 0.05$ 的 $\ell_\infty$ 对抗扰动。

In [48]:
epsilon = 0.1
initial_bound = ((X[0:1] - epsilon).clamp(min=0), (X[0:1] + epsilon).clamp(max=1))
bounds = bound_propagation(model_small, initial_bound)

for y_targ in range(10):
    if y_targ != y[0].item():
        c = np.eye(10)[y[0].item()] - np.eye(10)[y_targ]
        prob, _ = form_milp(model_small, c, initial_bound, bounds)
        print("Targeted attack {} objective: {}".format(y_targ, prob.solve(solver=cp.GUROBI)))

Targeted attack 0 objective: 0.774900466111629
Targeted attack 1 objective: 6.26327381581288
Targeted attack 2 objective: -3.225911939123418
Targeted attack 3 objective: -4.818509852618486
Targeted attack 4 objective: 5.186564397880513
Targeted attack 5 objective: 1.9734883186947645
Targeted attack 6 objective: 13.013704428222562
Targeted attack 8 objective: -2.256757870740434
Targeted attack 9 objective: 0.1290756386361407


在以 $\epsilon = 0.1$ 为半径的扰动球中，存在以 “2”，“3”，“8” 为目标样本的对抗性样本。

# 基于凸松弛求解目标函数上界

一个很显然的问题是，基于整数规划的内部最大化问题的精确解虽然很有研究价值，但是这种方法无法扩展到更大的网络，无论有多少计算资源，即使对中等规模的网络都很容易找到永远不会优化完成的问题。

因此上述的可验证鲁棒性并不够实用，我们需要快速获得内部最大化问题的上界，以此提供鲁棒性的正式保证。

例如，如果我们能获得一个上界，该上界仍然表明没有有针对性的攻击可以改变类别标签，这也提供了一种验证，即不可能有对抗样本。

在本节中，我们将介绍两种不同的形成上界的方法，一种是基于整数规划问题的凸松弛（提供一个更紧的界，但计算成本仍然较高），另一种基于界传播（提供了一个较松的界，但计算速度要快许多）。

## 基于整数规划问题的凸松弛

上述的整数规划问题求精确解的方法，计算成本主要高在引入的二进制约束 $v_i \in \{0, 1\}^{|v_i|}$, 用于精确捕获 ReLu 运算符。

如果去掉这个约束，即可从混合整数线性规划问题变为线性规划问题，从而快速求解。

因此，考虑对此二进制约束进行松弛处理，将元素取值从 $\{0, 1\}$ 放宽到 $[0, 1]$, 即可以取中间分数值。

除此之外，其他约束条件保持不变，这被称为整数问题的凸松弛。

对于松弛版本的优化攻击，如果求解的值仍然是正的，那么不存在对抗攻击，这是因为松弛集合严格大于原始集合，所以如果在松弛集合中不存在对抗样本，则原始集合也一定不存在。

我们可以使用如下方法来简单地描述这种松弛，对于原始的 ReLu 约束，它的所有取值是有界的，即在如下图中的向量取值中：
<p align="center">
<img src="./src/bounded_relu.svg" style="center">
<center>有上下界的 ReLU 函数</center>
</p>

而当我们将 $v_i$ 松弛为分数值时，结果证明这相当于将这个有界 ReLu 集合松弛为它的凸包：

<p align="center">
<img src="./src/relu_hull.svg" style="center">
<center>有上下界的 ReLU 凸包</center>
</p>


换句话说，松弛后的"经过ReLU后"的值 $z_{i+1}$ 不必是真正的 ReLU 值 $ReLU(W_iz_i+b_i)$ 。举例来说，对于松弛后的表达式，经过 ReLU 前的值可能是负的，但是对应的 $z_{i+1}$ 是正的。这意味着，如果我们将经过凸松弛后线性规划的**解**放入神经网络中，它大概率**不会**得到相同的结果。因此，**经过松弛之后得到的结果本身并不是一个对抗样本本身，而只是这个松弛后的优化问题的解。它可以证明*没有*对抗样本存在，但不能证明*一定存在*对抗样本**。由于外边界的特性，可能存在一种情况，**并不存在**对抗样本，但是松弛后的问题得到了一个解：由于放宽了 ReLU 限制，松弛问题可以得到一个将目标函数变为负数的解，尽管真正的样本无法做到这一点。而且，对于经过标准训练得到的网络，这个边界过于松弛；但是有意思的是，对于专门训练以减小这些上界的网络，我们经常*可以*提供非平凡的鲁棒性保证。

（大致意思是，若松弛后的线性规划问题得到的解可以使目标函数降至负数，并不代表找到了真正的对抗样本（因为松弛问题可能违背了 ReLU 的约束）；但是若松弛后的线性规划问题没有得到解，则**可以**说明不存在对抗样本，因为松弛后的问题是包含原来的约束问题的）

可以从另外一个视角观察探讨这个问题。当我们将 $l_{\infty}$ 球空间放入一个神经网络中，最后一层的结果是一个糟糕的、非凸的集合（对抗多面体）。

<p align="center">
<img src="./src/adversarial_polytope.svg" style="center">
<center>无穷范数约束下的输入，以及对应的对抗多面体</center>
</p>

在这个集合中找到一个最坏的点（一个朝着最大化目标标签 logits 值且最小化真标签 logits 值的方向移动最大的点）正是我们之前提到的 MILP 做的事情。但是如果我们考虑凸松弛后的这个集合，我们在考虑一个对抗多面体的*凸外边界*，这会使得优化更加高效。

<p align="center">
<img src="./src/adversarial_outer.svg" style="center">
<center>对抗多面体的凸外边界</center>
</p>


这个凸集合更容易优化（因为它是凸的），并且它是个严格更大的集合，所以如果在更大的集合中不存在对抗样本，则真正的对抗多面体中也不存在对抗样本。这个外边界在某些情况下可能非常松，但是对于大一点的深度神经网络来说这几乎是唯一可用的，得到强鲁棒性保证的方法了。

我们接下来将看看凸松弛后的约束问题是如何工作的。我们将使用和上文整数规划几乎一样的代码，不过我们将 $v_i$ 从一个 0-1 变量松弛到了一个值位于 0 到 1 之间的变量。

In [49]:
def form_lp(model, c, initial_bounds, bounds):
    linear_layers = [(layer, bound) for layer, bound in zip(model,bounds) if isinstance(layer, nn.Linear)]
    d = len(linear_layers)-1
    
    # create cvxpy variables
    z = ([cp.Variable(layer.in_features) for layer, _ in linear_layers] + 
         [cp.Variable(linear_layers[-1][0].out_features)])
    v = [cp.Variable(layer.out_features) for layer, _ in linear_layers[:-1]]
    
    # extract relevant matrices
    W = [layer.weight.detach().cpu().numpy() for layer,_ in linear_layers]
    b = [layer.bias.detach().cpu().numpy() for layer,_ in linear_layers]
    l = [l[0].detach().cpu().numpy() for _, (l,_) in linear_layers]
    u = [u[0].detach().cpu().numpy() for _, (_,u) in linear_layers]
    l0 = initial_bound[0][0].view(-1).detach().cpu().numpy()
    u0 = initial_bound[1][0].view(-1).detach().cpu().numpy()
    
    # add ReLU constraints
    constraints = []
    for i in range(len(linear_layers)-1):
        constraints += [z[i+1] >= W[i] @ z[i] + b[i], 
                        z[i+1] >= 0,
                        cp.multiply(v[i], u[i]) >= z[i+1],
                        W[i] @ z[i] + b[i] >= z[i+1] + cp.multiply((1-v[i]), l[i]),
                        v[i] >= 0,
                        v[i] <= 1]
    
    # final linear constraint
    constraints += [z[d+1] == W[d] @ z[d] + b[d]]
    
    # initial bound constraints
    constraints += [z[0] >= l0, z[0] <= u0]
    
    return cp.Problem(cp.Minimize(c @ z[d+1]), constraints), (z, v)

In [51]:
initial_bound = ((X[0:1] - epsilon).clamp(min=0), (X[0:1] + epsilon).clamp(max=1))
bounds = bound_propagation(model_small, initial_bound)
c = np.eye(10)[y[0].item()] - np.eye(10)[2]

prob, (z, v) = form_lp(model_small, c, initial_bound, bounds)
print("Objective value:", prob.solve(solver=cp.GUROBI))
print("Last layer from relaxation:", z[3].value)

Objective value: -30.445467280811798
Last layer from relaxation: [ 10.61930755  -8.00783511  24.99608703   7.27186848 -14.09313748
   0.36844093   0.42339702  -5.44938025   6.15953599 -12.38435955]


所以松弛后的解认为，我们可以使得类别 2 比真实类别的 logits 大 Objective value 这么多。

但是和整数规划的情况不一样，如果我们真的将这个**解出来的扰动**放入神经网络中，它的最后一层输出并不和松弛问题解出来的一致（但是在这个例子中，松弛问题依旧制造了一个对抗样本，类别 2 的预测值比真实类别 7 要大）。

In [53]:
print("Last layer from model:", 
      model_small(torch.tensor(z[0].value).float().view(1,1,28,28).to(device))[0].detach().cpu().numpy())

Last layer from model: [ 1.7471993  -2.5636814   8.261734    6.485653   -7.659953   -0.41698933
 -9.980528    6.367086    4.2566605  -2.2540815 ]


最后，我们来验证在面对很小扰动的情况下，网络的鲁棒性。这里和之前使用的代码大体一致，只不过调用的是凸松弛后的优化问题而不是整数规划的优化问题。

In [60]:
epsilon = 0.02
initial_bound = ((X[0:1] - epsilon).clamp(min=0), (X[0:1] + epsilon).clamp(max=1))
bounds = bound_propagation(model_small, initial_bound)

for y_targ in range(10):
    if y_targ != y[0].item():
        c = np.eye(10)[y[0].item()] - np.eye(10)[y_targ]
        prob, _ = form_lp(model_small, c, initial_bound, bounds)
        print("Targeted attack {} objective: {}".format(y_targ, prob.solve(solver=cp.GUROBI)))

Targeted attack 0 objective: 9.18712567594846
Targeted attack 1 objective: 14.61376613199486
Targeted attack 2 objective: 4.746316607451355
Targeted attack 3 objective: 3.8486686291440293
Targeted attack 4 objective: 14.239985579120846
Targeted attack 5 objective: 9.830999959024231
Targeted attack 6 objective: 23.256838783453667
Targeted attack 8 objective: 7.258987973138355
Targeted attack 9 objective: 7.865875866040316


所以我们可以得出结论：在 $\epsilon=0.02$ 的 $l_{\infty}$ 扰动下，没有对抗样本能成功欺骗这个分类器。而且和整数规划的情况不一样，我们可以在更大的模型上运行这个验证程序（它并不会真正提供一个整数规划的解）。下面我们可以在 4 层的 DNN 网络上运行它，而上面的整数规划程序并不能解出这个问题。

In [63]:
epsilon = 0.01
initial_bound = ((X[0:1] - epsilon).clamp(min=0), (X[0:1] + epsilon).clamp(max=1))
bounds = bound_propagation(model_dnn_4, initial_bound)

for y_targ in range(10):
    if y_targ != y[0].item():
        c = np.eye(10)[y[0].item()] - np.eye(10)[y_targ]
        prob, _ = form_lp(model_dnn_4, c, initial_bound, bounds)
        print("Targeted attack {} objective: {}".format(y_targ, prob.solve(solver=cp.ECOS)))

Targeted attack 0 objective: 4.9517729787365194
Targeted attack 1 objective: 3.874218213776362
Targeted attack 2 objective: 1.6757972500063651
Targeted attack 3 objective: -2.2969073548126184
Targeted attack 4 objective: 12.951596991011728
Targeted attack 5 objective: 0.140134351077827
Targeted attack 6 objective: 13.050637176316002
Targeted attack 8 objective: -1.4000302652154044
Targeted attack 9 objective: -2.093556869235737


## 凸松弛的更快解法

凸松弛已经比“慢”的整数规划问题更“快”了。但是上述代码在*一个* MNIST 样本上运行需要几秒钟，如果想在整个 MNIST 数据集上进行验证，而且在稍微更大一点的模型上运行，它的速度就不是很理想了。

所以，绝大部分关于使用凸松弛验证的方法都是基于*快速解决*（可能是近似解决，但是理想情况下仍然给出了一些保证）凸松弛问题。事实证明使用对偶方法，加上一些对原优化问题的操作，我们可以使用**一个简单的沿着网络的反向传播**，计算出关于松弛后的目标函数的一个可证明的下界（这给了一个原问题更“松”的松弛）。相较于上文中简单的边界传播算法，我们可以使用这个非常快的方法来计算一个更紧的间隔边界(interval bounds)。然而，验证程序的复杂度和边界的严格程度(tightness)之间的权衡仍然是一个开放的研究问题，而且目前没有真正的可扩展的(scalable)解决方案（一个既计算高效又可以对大规模网络提供紧边界）。

## 基于间隔传播的边界(interval-propagation-based bound)
这里提供一个得到该优化问题上界的一个方法，这个方法直接基于上文提到过的边界传播方法。这个方法得到的边界比凸松弛得到的更弱，并且这个方法甚至不能和凸松弛一样提供一个“假的”对抗样本，但它的优点是它**效率特别高**。最近的研究工作表明，在对抗训练中，更应该将这些边界传播技术与更复杂的网络结合，而不是将基于凸松弛的更严格但是花费时间更多的边界与更简单的网络结合。不过两种方法的全面比较仍然是一个活跃的研究课题。

我们再回到之前探讨过的，通过网络传播间隔边界的方法。假设对于某一层的输入 $x$ 有上下界 $\hat{l} \leq x \leq \hat{u}$ ，则对于该层的输出有
$$
l\leq Wx+b \leq u
$$
其中
$$
l=\max\{W,0\}\hat{l}+\min\{W,0\}\hat{u}+b \\
u=\min\{W,0\}\hat{l}+\max\{W,0\}\hat{u}+b
$$
对于一个 $d$ 层的网络，对抗攻击的目标是**最小化**关于最后一层输出 $z_{d+1}$ 的“某种”线性函数 $c^Tz_{d+1}$ (可以是单位向量 $e_y$ ，其只有第 $y$ 位为1其余都为0)。假设倒数第二层有上下界 $\hat{l}\leq z_d\leq \hat{u}$ ，那么对抗攻击对应的优化问题转化为
$$
\min c^Tz_{d+1} \\
\text{ s.t. } z_{d+1}=W_dz_d+b_d \\
\hat{l}\leq z_d \leq \hat{u}
$$
该问题可以转化为以下等价形式
$$
\min c^T(W_dz_d+b_d)=(c^TW_d)z_d+c^Tb_d \\
\text{ s.t. } \hat{l}\leq z_d \leq\hat{u}
$$
该优化问题的解为
$$
\max\{c^TW_d,0\}\hat{l}+\min\{c^TW_d,0\}\hat{u}+c^Tb_d
$$

接下来我们看看这个方法与我们之前的边界传播算法结合的效果。注意到这个方法的其中一个好处是，我们很容易就写下了同时计算多个 $c$ （当我们考虑多目标攻击时）与小批次中多个样本的边界的代码。 

In [64]:
def interval_based_bound(model, c, bounds):
    # requires last layer to be linear
    cW = c.t() @ model[-1].weight
    cb = c.t() @ model[-1].bias
    l,u = bounds[-2]
    return (cW.clamp(min=0) @ l.t() + cW.clamp(max=0) @ u.t() + cb[:,None]).t()  

我们将这个得到的边界应用到 $\epsilon=0.02$ 的情况下。我们之前已经使用凸松弛验证过这个扰动范围。

In [65]:
epsilon = 0.02
initial_bound = ((X[0:1] - epsilon).clamp(min=0), (X[0:1] + epsilon).clamp(max=1))
bounds = bound_propagation(model_small, initial_bound)
C = -torch.eye(10).to(device)
C[y[0].item(),:] += 1

print(interval_based_bound(model_small, C, bounds).detach().cpu().numpy())

[[-5.2146845  0.9834416 -8.95515   -9.430425  -1.6530738 -5.675324
   6.744165   0.        -9.852957  -7.0858884]]


这里输出的每一项是目标函数对于每一个目标攻击的下界（下标从 0 开始，注意到第 7 类的下界是 0 ，因为 7 就是真实样本，对应的 $c=0$ ）。这些下界，比我们用凸松弛得到的下界更宽松。和凸松弛不同，并没有一个输入（即使是在松弛后的 ReLU 运算下）可以达到这个目标类别 logits 减真类别 logits 的下界：每一层激活层的优化都假定前一层的激活层可以*仅*取部分离散的值从而仅使部分激活元达到最大或最小值。但是这个方法得到的边界有一个极大的优点，即它们计算起来非常快，而且已经完全集成在 Pytorch 库里了。这意味着我们可以通过它们进行反向传播来基于某个鲁棒准则(robust criterion)训练鲁棒的对抗网络。